# Zenith Scientific Test 2: The Bitwise Investigator

**Objective:**
Verify that Zenith's speedups do not come at the cost of "Silent Errors". We will compare the output of specific mathematical operators against PyTorch native execution, bit-by-bit.

**Methodology:**
1.  Generate random input tensors (including edge cases with large magnitudes).
2.  Pass them through standard layers: `Softmax`, `LayerNorm`, `GELU`.
3.  Execute once with `torch.compile(backend='zenith')` and once with Native PyTorch.
4.  **Metric:** Calculate the **Maximum Absolute Difference** (`abs(zenith - torch).max()`).

**Success Criteria:**
*   Difference should be negligible (< `1e-5` for FP32/FP16 mixed).
*   Zenith must match PyTorch's numerical behavior.

In [ ]:
!pip install -q -U pyzenith torch numpy

import torch
import torch.nn as nn
import zenith
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

def compare_layer(layer_name, layer, input_tensor):
    print(f"\n--- Testing {layer_name} ---")
    
    # 1. Native PyTorch Run
    layer.eval()
    with torch.no_grad():
        out_native = layer(input_tensor)
    
    # 2. Zenith Run
    # We re-create the layer or compile the existing one
    # Ideally, we verify component-level compilation
    opt_layer = torch.compile(layer, backend="zenith")
    
    # Warmup
    with torch.no_grad():
        _ = opt_layer(input_tensor)
        
    # Actual Run
    with torch.no_grad():
        out_zenith = opt_layer(input_tensor)
    
    # 3. Analysis
    diff = torch.abs(out_native - out_zenith)
    max_diff = diff.max().item()
    mean_diff = diff.mean().item()
    
    print(f"Max Diff: {max_diff:.9f}")
    print(f"Mean Diff: {mean_diff:.9f}")
    
    if max_diff < 1e-4:
        print(f"PASSED: {layer_name} is precise.")
    else:
        print(f"FAILED: {layer_name} shows significant divergence!")
        
    return max_diff

### Test Case 1: Softmax (Sensitivity Test)
Softmax involves exponentials, which are very sensitive to optimization reordering.

In [ ]:
input_softmax = torch.randn(1024, 1024, device=device, dtype=torch.float32)
layer_softmax = nn.Softmax(dim=-1)

compare_layer("Softmax (FP32)", layer_softmax, input_softmax)

### Test Case 2: LayerNorm (Fusion Test)
LayerNorm is often fused by compilers. Bad fusion leads to wrong variance calculation.

In [ ]:
input_ln = torch.randn(512, 768, device=device, dtype=torch.float32)
layer_ln = nn.LayerNorm(768).to(device)

compare_layer("LayerNorm", layer_ln, input_ln)

### Test Case 3: GELU (Approximation Test)
GELU often uses `tanh` approximation. We need to check if Zenith's approximation matches PyTorch's.

In [ ]:
input_gelu = torch.randn(1024, 1024, device=device, dtype=torch.float32)
layer_gelu = nn.GELU()

compare_layer("GELU", layer_gelu, input_gelu)